# Web Crawler: [spotifycharts.com](https://spotifycharts.com/)
## Contents:
**0. [Imports](#0.-Imports)**  
**1. [Helper Methods](#1.-Helper-Methods)**  
**2. [Chart Crawling and Scraping Methods](#2.-Chart-Crawling-and-Scraping-Methods)**  
**3. [Method Call](#3.-Method-Call)**  
**4. [Testing](#4.-Testing)**  

---
## 0. Imports

In [1]:
from bs4 import BeautifulSoup
import http.client
import urllib
import os

import pandas as pd
import numpy as np
import csv

import dateutil.parser
import datetime
import sys

---
## 1. Helper Methods

In [2]:
def det_trend_type(trend_fill:str) -> str:
    """Returns the trend type given the fill color of the Spotify Chart trend SVG."""
    # i. color: green; shape: up-triangle
    if trend_fill == '#84bd00':
        return 'Up'
    # ii. color: gray; shape: horizontal-rectangle
    elif trend_fill == '#3e3e40':
        return 'Flat'
    # iii. color: red; shape: down-triangle
    elif trend_fill == '#bd3200':
        return 'Down'
    # iv. color: blue; shape: circle
    elif trend_fill == '#4687d7':
        return 'New'
    # v. unhandled: non-v.1.1 tend_fill
    else:
        return 'UNKNOWN'

In [3]:
def get_datestamp_str(date):
    """Returns the current datestamp as a string in YY-MM-DD format."""
    datestamp_str = date.strftime('%Y-%m-%d')
    return datestamp_str

In [4]:
def get_timestamp_str():
    """Returns the current timestamp as a string in YY-MM-DD HH-SS-FF format."""
    now_datetime = datetime.datetime.now()
    timestamp_str = now_datetime.strftime('D-%Y-%m-%d_T-%H-%M-%S-%f')
    return timestamp_str

In [5]:
def get_soup(url):
    """Returns the BeautifulSoup object for the given URL."""
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    url_req = urllib.request.Request(url, headers = {'User-Agent': user_agent})
    req_resp = urllib.request.urlopen(url_req)
    req_read = req_resp.read()
    req_resp.close()
    return BeautifulSoup(req_read)

In [6]:
def scrape_region_dict(chart_type_url):
    """Returns a dictionary of region abbreviation keys and region name values."""
    chart_soup = get_soup(chart_type_url)
    region_li_tags = chart_soup.find('div', 
                                     attrs={'class':'responsive-select','data-type':'country'}).find_all('li')
    region_dict = {}
    for li_tag in region_li_tags:
        region_abrv = li_tag.get('data-value').strip()
        region_name = li_tag.get_text()
        region_dict[region_abrv] = region_name
    
    return region_dict

In [7]:
def scrape_charts_dates(region_url, strt_date_arg, end_date_arg):
    """Returns a list of date strings representing the available charts."""
    region_soup = get_soup(region_url)
    date_div = region_soup.find('div', attrs={'class':'responsive-select','data-type':'date'})
    
    if date_div is None:
        return np.array([], dtype=datetime.date)
    
    else:
        date_li_tags = date_div.find_all('li')
        
        first_li_tag = date_li_tags[0]
        first_li_date = dateutil.parser.isoparse(first_li_tag.get('data-value').strip()).date()
        max_date = first_li_date if end_date_arg is None else end_date_arg

        last_li_tag = date_li_tags[-1]
        last_li_date = dateutil.parser.isoparse(last_li_tag.get('data-value').strip()).date()
        min_date = last_li_date if strt_date_arg is None else strt_date_arg
        
        num_days_delta = (max_date - min_date).days
        date_range_list = [min_date + datetime.timedelta(days=x) for x in range(num_days_delta + 1)]
    
        return date_range_list

In [8]:
def handle_error(chart_date, chart_region_abrv, chart_name, idx , chart_date_url, error_code):
    """Returns a list of and out prints error details."""
    print('ERROR  : ',  'attempt ', idx, ' ; ', error_code)
    print('         ', chart_date, ' - ', chart_region_abrv, ' - ', chart_name, ' - ', chart_date_url)
    return [chart_name, chart_region_abrv, chart_date, chart_date_url, idx, str(error_code)]

In [23]:
def gen_csv_filepath(type_str, now_timestamp, start_datestamp, end_datestamp):
    dir_path = './data/'+now_timestamp
    if not os.path.exists(dir_path): os.mkdir(dir_path)
    
    file_name = type_str + now_timestamp + '_FrmD-' + start_datestamp + '_ToD-' + end_datestamp + '.csv'
    file_path = os.path.join(dir_path, file_name)
    
    return file_path

In [24]:
def gen_analysis_report(result_df_arg, error_df_arg):
    """Prints analysis metrics for the chart crawl."""
    print('\nRESULT ANALYSIS')
    
    result_analysis_dict = {
        'Num Rows' : result_df_arg.shape[0],
        'Num Cols' : result_df_arg.shape[1],
        'Num Unique Dates' : len(result_df_arg['Date'].unique()),
        'Num Unique Charts' : len(result_df_arg['Chart'].unique()),
        'Num Unique Regions' : len(result_df_arg['Region'].unique()),
        'Num Unique Positions' : len(result_df_arg['Position'].unique()),
        'Num Unique Trends' : len(result_df_arg['Trend'].unique()),
        'Num Unique Titles' : len(result_df_arg['Title'].unique()),
        'Num Unique Artists' : len(result_df_arg['Artist'].unique()),
        'Num Unique Icon_URLs' : len(result_df_arg['Icon_URL'].unique()),
        'Num Unique Spotify_URLs' : len(result_df_arg['Spotify_URL'].unique())
    }
    
    result_analysis_df = pd.DataFrame(data=list(result_analysis_dict.values()), columns=['Value'])
    result_analysis_df['Metric'] = list(result_analysis_dict.keys())
    result_analysis_df.set_index('Metric', inplace=True)
    
    rows_equals_dates = (result_analysis_dict['Num Rows'] 
                         == result_analysis_dict['Num Unique Dates']*200)
    titles_equals_urls = (result_analysis_dict['Num Unique Titles'] 
                          == result_analysis_dict['Num Unique Spotify_URLs'])
    
    min_date = result_df_arg['Date'].min()
    max_date = result_df_arg['Date'].max()
    delta_days = (max_date - min_date).days
    date_equals_delta = (len(result_df_arg['Date'].unique()) == delta_days)

    print('\n-----\n')
    print('RESULTS:\n')
    print(result_analysis_df)
    print('\nNum Rows==Dates?')
    print(' – YES' if rows_equals_dates else ' – NO: ' + str(result_analysis_dict['Num Rows']) 
          + ' ≠ ' + str(result_analysis_dict['Num Unique Dates']*200) 
          + ' = ' + str(result_analysis_dict['Num Unique Dates']) + ' * 200')
    print('\nNum Titles==Spotify_URLs?')
    print(' – YES' if titles_equals_urls else ' – NO: '+ str(result_analysis_dict['Num Unique Titles']) 
          + ' ≠ ' + str(result_analysis_dict['Num Unique Spotify_URLs']))
    print('\nNum Dates==Date Delta?')
    print(' – YES' if date_equals_delta else ' – NO: '+ str(len(result_df_arg['Date'].unique())) 
          + ' ≠ ' + str(delta_days))

    
    # Error Col Names:
    # 'Chart', 'Region', 'Date', 'Chart URL', 'Attempt', 'Error'
    
    incomp_read_df = error_df_arg[error_df_arg['Error'].str.startswith('IncompleteRead')]
    http_df = error_df_arg[error_df_arg['Error'].str.startswith('HTTP Error')]
    skipped_df = error_df_arg[error_df_arg['Attempt']==-1]
    combined_df = (error_df_arg['Error'].str.startswith('IncompleteRead', na=True) | 
                   error_df_arg['Error'].str.startswith('HTTP Error', na=True))
    
    error_analysis_dict = {
        'Num IncompleteRead' : len(incomp_read_df),
        'Num HTTP' : len(http_df),
        'Num Other' : combined_df[combined_df==False].count(),
        'Num Total Errors' : len(error_df_arg),
    }
    
    error_analysis_df = pd.DataFrame(data=list(error_analysis_dict.values()), columns=['Value'])
    error_analysis_df['Metric'] = list(error_analysis_dict.keys())
    error_analysis_df.set_index('Metric', inplace=True)
    
    http_equals_skips = (error_analysis_dict['Num HTTP'] == len(skipped_df))
    
    error_sum = (error_analysis_dict['Num IncompleteRead'] 
                 + error_analysis_dict['Num HTTP'] 
                 + error_analysis_dict['Num Other'])
    
    total_equals_sum = error_analysis_dict['Num Total Errors'] == error_sum
    
    print('\n-----\n')
    print('ERRORS:\n')
    print(error_analysis_df)
    print('\nNum HTTP==Skips?')
    print(' – YES' if http_equals_skips else ' – NO: ' + str(error_analysis_dict['Num HTTP']) 
          + ' ≠ ' + str(len(skipped_df)))
    print('\nNum Total==Sum?')
    print(' – YES' if total_equals_sum else ' – NO: '+ str(error_analysis_dict['Num Total Errors']) 
          + ' ≠ ' + str(error_sum)
          + ' = (' + str(error_analysis_dict['Num IncompleteRead']) 
          + ' + ' + str(error_analysis_dict['Num HTTP']) 
          + ' + ' + str(error_analysis_dict['Num Other']) + ' )')
    print('\nMin date:', min_date)
    print('Max date:', max_date)
    print('\nSKIPPED CHARTS:\n')
    print('\n')
    print(skipped_df)

---
## 2. Chart Crawling and Scraping Methods

In [25]:
def scrape_chart(date_soup, chart_name, chart_region, chart_date):
    """Returns pd/DataFrame of data scraped from each chart URL's <table class="chart-table" ...> tag"""
    tr_tags = date_soup.find('table', class_='chart-table').find('tbody').find_all('tr')
    chart_data = []
    
    for tr_tag in tr_tags:
        
        song_info = []
        
        # 0. col1, col2, col3: Chart Name, Chart Region, and Chart Date
        song_info.append(chart_name)
        song_info.append(chart_region)
        song_info.append(chart_date)
        
        # 1. col4: Chart Position; element of {1, 2, ... , 200}
        pos_tag = tr_tag.find('td', class_='chart-table-position')
        pos_val = int(pos_tag.get_text().strip())
        song_info.append(pos_val)
        
        # 2. col5: Streaming Trend Type; element of {Up, Flat, Down, New}
        trend_tag = tr_tag.find('td', class_='chart-table-trend')
        trend_fill = trend_tag.find('svg').get('fill').strip()
        trend_val = det_trend_type(trend_fill)
        song_info.append(trend_val)
        
        # 3. col6, col7: Track Title and Artist
        track_tag = tr_tag.find('td', class_='chart-table-track')
        title_val = track_tag.find('strong').get_text().strip()
        artist_val = track_tag.find('span').get_text().strip().replace('by ', '')
        song_info.append(title_val)
        song_info.append(artist_val)
        
        # 4. col8: Total Daily Streams
        if chart_name == 'Viral50':
            song_info.append(None)
        else:
            streams_tag = tr_tag.find('td', class_='chart-table-streams')
            streams_val = int(streams_tag.get_text().strip().replace(',',''))
            song_info.append(streams_val)
        
        # 5. col9, col10: Icon and Spotify URLs
        icon_tag = tr_tag.find('td', class_='chart-table-image')
        icon_url_val = icon_tag.find('img').get('src').strip()
        spotify_url_val = icon_tag.find('a').get('href').strip()
        song_info.append(icon_url_val)
        song_info.append(spotify_url_val)
        
        #6. col11: Spotify ID
        spotify_id_val = spotify_url_val[(spotify_url_val.rfind('/') + 1):].strip()
        song_info.append(spotify_id_val)
        
        chart_data.append(song_info)

    return chart_data

In [31]:
def crawl_spotify_charts(start_date=None,
                         end_date=None,
                         select_regions_dict=None,
                         crawl_viral=False,
                         crawl_weekly=False,
                         colab_save=False):
    """Returns a pd.DataFrame containing information scraped from https://spotifycharts.com."""
    # 1. loop through charts
    base_url = 'https://spotifycharts.com'
    
    chart_type_dict = {'regional':'Top200', 'viral':'Viral50'} if crawl_viral else {'regional':'Top200'}
    chart_interval_list = ['daily', 'weekly'] if crawl_weekly else ['daily']
    
    # 2. instantiates result lists
    result_data = []
    error_list = []
    
    print('----------------------------------------------------------------------')
    print('\nWEB CRAWLING LOG:\n\n')

    # 3. scrapes all available charts
    # 3.1. loops through chart types
    for chart_type, chart_name in chart_type_dict.items():
        
        chart_type_url = base_url + '/' + chart_type
        if select_regions_dict is None:
            chart_regions_dict = scrape_region_dict(chart_type_url)
        else:
            chart_regions_dict = select_regions_dict

        # 3.2. loops through chart regions
        for chart_region_abrv, chart_region_name in chart_regions_dict.items():

            # 3.3. loops through chart intervals
            for chart_interval in chart_interval_list:
                
                latest_charts_url = chart_type_url + '/' + chart_region_abrv + '/' + chart_interval

                try:
                    charts_dates_list = scrape_charts_dates(latest_charts_url, start_date, end_date)
                    
                except (http.client.IncompleteRead, urllib.error.HTTPError) as err:
                    error_list.append(handle_error(chart_date, chart_region_abrv, 
                                                   chart_name, np.nan , chart_date_url, err))
                    continue
                
                # 3.4. loops through all available chart dates
                for chart_date in charts_dates_list:

                    chart_date_url = latest_charts_url + '/' + chart_date.isoformat()

                    # 3.4.1. tries, then retries 3 times, to read the chart's html page
                    for attempt in range(1,4):
                        # 3.4.1.1. tries to read the url, and if successful, scrapes and appends its data
                        try:
                            chart_date_soup = get_soup(chart_date_url)
                            chart_date_data = scrape_chart(chart_date_soup, 
                                                           chart_name, 
                                                           chart_regions_dict[chart_region_abrv], 
                                                           chart_date)
                            result_data.append(chart_date_data)
                            print('Success: ', chart_date, 
                                  ' - ', chart_region_abrv, 
                                  ' - ', chart_name, 
                                  ' - ', chart_date_url)
                            break

                        # 3.4.1.2. cathces IncompleteRead exceptions, retrying the URL twice
                        except (http.client.IncompleteRead, urllib.error.HTTPError) as err:
                            error_list.append(handle_error(chart_date, chart_region_abrv, 
                                                           chart_name, attempt , chart_date_url, err))
                            continue

                    # 3.5. otherwise, if the loop falls through, skips this chart date
                    else:
                        error_list.append(handle_error(chart_date, chart_region_abrv, 
                                                       chart_name, -1 , chart_date_url, 'SKIPPED'))
                        continue

    print('\n----------------------------------------------------------------------')
    print('\n\tCOMPLETED: WEB CRAWLING/SCRAPING')
                        
    # 4. creates both the result and error pd.DataFrames from the accumulated list of data entries
    result_data_flat = [chart_entry for date_data in result_data for chart_entry in date_data]
    result_df_cols = ['Chart', 'Region' , 'Date', 'Position', 'Trend', 
                      'Title', 'Artist', 'Streams', 'Icon_URL', 'Spotify_URL', 'Spotify_ID']
    result_df = pd.DataFrame(result_data_flat, columns=result_df_cols)
    
    error_df_cols = ['Chart', 'Region', 'Date', 'Chart URL', 'Attempt', 'Error']
    error_df = pd.DataFrame(error_list, columns=error_df_cols)                 
    
    # 5. saves both the result and error pd.DataFrame as csv files  
    now_timestamp = get_timestamp_str()
    start_datestamp = get_datestamp_str(charts_dates_list[0])
    end_datestamp = get_datestamp_str(charts_dates_list[-1])
    
    result_df.to_csv(gen_csv_filepath('Result_', now_timestamp, start_datestamp, end_datestamp))
    
    top200_df = result_df[result_df['Chart']=='Top200']
    top200_df.to_csv(gen_csv_filepath('Top200_', now_timestamp, start_datestamp, end_datestamp))
    
    viral50_df = result_df[result_df['Chart']=='Viral50']
    viral50_df.to_csv(gen_csv_filepath('Viral50_', now_timestamp, start_datestamp, end_datestamp))
    
    error_df.to_csv(gen_csv_filepath('Errors_', now_timestamp, start_datestamp, end_datestamp))
    
    print('\n\tCOMPLETED: DATAFRAME/CSV CREATION')
    print('\n----------------------------------------------------------------------')
    gen_analysis_report(result_df, error_df)
    print('\n----------------------------------------------------------------------')
    
    # 6. returns the result pd.DataFrame
    return result_df

---
## 3. Method Call

In [ ]:
# WEB CRAWL CALL

# date_var1 = datetime.date(2017,1,1)
# date_var2 = datetime.date(2017,1,5)
# crawl_df = crawl_spotify_charts(
#     start_date=date_var1,
#     end_date=date_var2,
#     select_regions_dict={'us':'United States'}, 
#     crawl_viral=True)

crawl_df = crawl_spotify_charts(
    start_date=None,
    end_date=None,
    select_regions_dict={'us':'United States'}, 
    crawl_viral=True)

crawl_df

----------------------------------------------------------------------

WEB CRAWLING LOG:


Success:  2017-01-01  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-01-01
Success:  2017-01-02  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-01-02
Success:  2017-01-03  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-01-03
Success:  2017-01-04  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-01-04
Success:  2017-01-05  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-01-05
Success:  2017-01-06  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-01-06
Success:  2017-01-07  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-01-07
Success:  2017-01-08  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-01-08
Success:  2017-01-09  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-01-09
Success:  2017-01-10  -  u

Success:  2017-03-25  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-03-25
Success:  2017-03-26  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-03-26
Success:  2017-03-27  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-03-27
Success:  2017-03-28  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-03-28
Success:  2017-03-29  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-03-29
Success:  2017-03-30  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-03-30
Success:  2017-03-31  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-03-31
Success:  2017-04-01  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-04-01
Success:  2017-04-02  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-04-02
Success:  2017-04-03  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-04-03
Success:  2017-04-04

Success:  2017-06-03  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-06-03
Success:  2017-06-04  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-06-04
Success:  2017-06-05  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-06-05
Success:  2017-06-06  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-06-06
Success:  2017-06-07  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-06-07
Success:  2017-06-08  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-06-08
Success:  2017-06-09  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-06-09
Success:  2017-06-10  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-06-10
Success:  2017-06-11  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-06-11
Success:  2017-06-12  -  us  -  Top200  -  https://spotifycharts.com/regional/us/daily/2017-06-12
Success:  2017-06-13

---
## 4. Testing

### 4.1. Test Variables

In [ ]:
# oneday_delta= datetime.timedelta(days = 1)

# jan2017_strt = datetime.date(2017,1,1)
# jan2017_5 = datetime.date(2017,1,5)
# feb2017_strt = datetime.date(2017,2,1)
# mar2017_strt = datetime.date(2017,3,1)

# jan2017_end = feb2017_strt - oneday_delta
# feb2017_end = mar2017_strt - oneday_delta

### 4.2. Test Method

In [ ]:
# # Test Call:
# crawl_spotify_charts(start_date = jan2017_strt, 
#                      end_date = mar2017_strt,
#                      select_regions_dict={'us':'United States'})